In [7]:
# start by importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# set up the grid
dx = 0.5
x = np.arange(1, 100, dx)

# some constants
c = 1.0
baselevel_rate = 0.01
Hstar = 1.0e-3

# k values
k_ig = 0.0001
k_sed = 0.001

# abrasion coeffs, in % per km
beta_ig = 0.004
beta_sed = 0.098

# percentages in basin
amt_ig = 1
amt_sed = 0.0

# time settings
num_steps = 1000

In [11]:
# a function
def two_lith_bedrock_and_abrasion(x,
                                  dx,
                                  c,
                                  Hstar,
                                  amt_ig,
                                  amt_sed,
                                  k_ig = 0.0001,
                                  k_sed = 0.001,
                                  beta_ig = 0.004,
                                  beta_sed = 0.098,  
                                  baselevel_rate=0.01, 
                                  porosity = 0.55, 
                                  num_steps = 1000):
    
    # create arrays of erodibilities
    k_ig = np.zeros(len(x))
    k_ig[:100] = k_ig
    k_sed = np.zeros(len(x))
    k_sed[100:] = k_sed
    
    # create arrays of bed abrasion coefficients
    bed_abrasion_ig = np.zeros(len(x))
    bed_abrasion_ig[:100] = beta_ig
    bed_abrasion_sed = np.zeros(len(x))
    bed_abrasion_sed[100:] = beta_sed
    
    # set sediment thickness
    H_ig = np.zeros(len(x))
    H_sed = np.zeros(len(x))
    H = Hstar + H_ig + H_sed
    H[-1] = 0
    
    # set bedrock elev
    etab = -H
    
    # set total elev
    eta = etab + H
    
    # create array to hold erosion
    ero = np.zeros(len(x))
    
    # set discharge as fxn of downstream distance
    q = x
    
    # create array to hold sediment flux (first node is left edge of 0th cell)
    qs = np.zeros(len(x))
    qs_ig = np.zeros(len(x))
    qs_sed = np.zeros(len(x))
    
    # create arrays to hold percentages of different lithologies
    theta = 1
    theta_ig = np.zeros(len(x))
    theta_sed = np.zeros(len(x))
    # theta_sed[:] = theta
    
    # set time step size
    dt = 0.2 * dx * dx / c

    for t in range(num_steps):
        
        # lower baselevel
        eta[-1] -= baselevel_rate * dt
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta) / dx
        
        # calc bed cover
        efac = np.exp(-H / Hstar)
        
        # calc bed sed flux
        qs[1:] = c * q[1:] * S * (1.0 - efac[:-1])
        qs[0] = 0
        
        # calculate flux of each lithology
        qs_ig[1:] = qs[1:] * (theta_ig[:-1])
        qs_sed[1:] = qs[1:] * (theta_sed[:-1])
        
        # set constant, mixed feed of grains
        qs_ig[0] = amt_ig
        qs_sed[0] = amt_sed
        
        # update percentages based on alluvial thickness
        undefined = np.where(H == 0)
        defined = np.where(H != 0)
        
        theta_ig[undefined] = 0
        theta_ig[defined] += H_ig[defined]/H[defined]
        
        theta_sed[undefined] = 0
        theta_sed[defined] += H_sed[defined]/H[defined]
        
        # conserve mass
        theta_total = theta_ig + theta_sed
        dtheta = theta - theta_total
        theta_ig += dtheta * theta_ig
        theta_sed += dtheta * theta_sed
        
        # calc bedrock erosion from plucking
        pluck_ig = efac[:-1] * (k_ig[1:] * q[1:] * S)
        pluck_sed = efac[:-1] * (k_sed[1:] * q[1:] * S)
        
        # calc bedrock erosion from abrasion
        abr_ig = efac[:-1] * (bed_abrasion_ig[:-1] * qs[:-1])
        abr_sed = efac[:-1] * (bed_abrasion_sed[:-1] * qs[:-1])
        
        # calc bedrock erosion rate from stream power and abrasion
        ero[:-1] = pluck_ig + pluck_sed + abr_ig + abr_sed
        
        # calc change in bedrock elev
        etab[:-1] -= ero[:-1] * dt
        
        # grain attrition
        ig_atr = beta_ig * qs[1:]
        sed_atr = beta_sed * qs[1:]

        # calc change in alluvial thickness
        H_ig[:-1] += -((1 / porosity) * ((np.diff(qs_ig)/dx) + ig_atr - pluck_ig)) * dt
        H_sed[:-1] += -((1 / porosity) * ((np.diff(qs_sed)/dx) + sed_atr - pluck_sed)) * dt
        
        H[:-1] += H_ig[:-1] + H_sed[:-1]
        
        # update elev
        eta[:-1] = etab[:-1] + H[:-1]

    return eta, etab, H_ig, H_sed, qs, pluck_ig, pluck_sed, abr_ig, abr_sed, atr_ig, atr_sed, theta_ig, theta_sed, x

In [12]:
eta, etab, H_ig, H_sed, qs, pluck_ig, pluck_sed, abr_ig, abr_sed, atr_ig, atr_sed, theta_ig, theta_sed, x = two_lith_bedrock_and_abrasion(x,
                                                                                                                                         dx,
                                                                                                                                         c,
                                                                                                                                         Hstar,
                                                                                                                                         amt_ig,
                                                                                                                                         amt_sed)

ValueError: could not broadcast input array from shape (198) into shape (100)